# Query Analysis

Analysis of queries in the database to understand distribution and uniqueness.

In [5]:
import os
import pandas as pd
import sys
from dotenv import load_dotenv

load_dotenv()
root_folder = os.environ.get("root_folder")
print(f"Root folder: {root_folder}")
sys.path.append(root_folder)

Root folder: /Users/luvsuneja/Documents/repos/masala-embed/esci-dataset


In [7]:
from database.utils.db_utils import get_db_connection

In [8]:
# Read all queries from the query table
with get_db_connection() as conn:
    query_df = pd.read_sql(
        """
        SELECT 
            id,
            query_content,
            query_filters,
            data_gen_hash,
            mlflow_run_id,
            created_at
        FROM query
        ORDER BY created_at DESC
    """,
        conn,
    )

print(f"Total records loaded: {len(query_df)}")
query_df.head()

/var/folders/jk/dyw0vdnx2jg9lyq8m01n8nfm0000gn/T/ipykernel_80367/1680117440.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  query_df = pd.read_sql("""


Total records loaded: 4957


,id,query_content,query_filters,data_gen_hash,mlflow_run_id,created_at
0,4709,Middle Eastern vegan friendly,None,5577177,aefc21fcce0440a28943f874c04dd37d,2025-09-24 18:16:35.174157+00:00
1,4708,Middle Eastern sharing plates,None,5577177,aefc21fcce0440a28943f874c04dd37d,2025-09-24 18:16:35.174157+00:00
2,4707,Korean food near me,None,5577177,aefc21fcce0440a28943f874c04dd37d,2025-09-24 18:16:35.174157+00:00
3,5311,wholesome family dinner,None,5577177,aefc21fcce0440a28943f874c04dd37d,2025-09-24 18:16:35.174157+00:00
4,5310,white-tablecloth vibes at home,None,5577177,aefc21fcce0440a28943f874c04dd37d,2025-09-24 18:16:35.174157+00:00


In [9]:
# Get total records and unique queries
total_records = len(query_df)
unique_queries = query_df["query_content"].nunique()

print(f"📊 Query Analysis Summary")
print(f"========================")
print(f"Total records: {total_records:,}")
print(f"Unique queries: {unique_queries:,}")
print(
    f"Duplicate rate: {((total_records - unique_queries) / total_records * 100):.2f}%"
)

📊 Query Analysis Summary
Total records: 4,957
Unique queries: 4,932
Duplicate rate: 0.50%


In [13]:
# Value counts for query content
query_value_counts = query_df["query_content"].value_counts()
(query_value_counts > 1).sum()

np.int64(25)

In [14]:
# Distribution of query frequencies
frequency_distribution = query_value_counts.value_counts().sort_index()

print(f"\n📊 Query Frequency Distribution:")
print(f"===============================\n")
for frequency, count in frequency_distribution.items():
    print(f"{count:4d} queries appear {frequency:2d} time(s)")


📊 Query Frequency Distribution:

4907 queries appear  1 time(s)
  25 queries appear  2 time(s)


In [15]:
# Analysis by MLflow run
mlflow_analysis = (
    query_df.groupby("mlflow_run_id")
    .agg({"id": "count", "query_content": "nunique", "created_at": ["min", "max"]})
    .round(2)
)

mlflow_analysis.columns = [
    "total_records",
    "unique_queries",
    "first_created",
    "last_created",
]
mlflow_analysis["duplicate_rate"] = (
    (mlflow_analysis["total_records"] - mlflow_analysis["unique_queries"])
    / mlflow_analysis["total_records"]
    * 100
).round(2)

print(f"\n🏃 Analysis by MLflow Run:")
print(f"=========================\n")
print(mlflow_analysis)


🏃 Analysis by MLflow Run:

                                  total_records  unique_queries  \
mlflow_run_id                                                     
0c611a39b1844107b10d4f7ac9282770           2772            2772   
3802814759d244af89aae038575f0202           1573            1573   
aefc21fcce0440a28943f874c04dd37d            612             612   

                                                    first_created  \
mlflow_run_id                                                       
0c611a39b1844107b10d4f7ac9282770 2025-09-24 17:36:29.503553+00:00   
3802814759d244af89aae038575f0202 2025-09-24 17:29:47.669335+00:00   
aefc21fcce0440a28943f874c04dd37d 2025-09-24 18:16:35.174157+00:00   

                                                     last_created  \
mlflow_run_id                                                       
0c611a39b1844107b10d4f7ac9282770 2025-09-24 17:36:29.503553+00:00   
3802814759d244af89aae038575f0202 2025-09-24 17:29:47.669335+00:00   
aefc21fcce0440

In [16]:
# Identify duplicate queries with their IDs
duplicate_queries = (
    query_df.groupby("query_content").agg({"id": ["count", "min", list]}).reset_index()
)

# Flatten column names
duplicate_queries.columns = ["query_content", "count", "first_id", "all_ids"]

# Filter only duplicates (count > 1)
duplicates_only = duplicate_queries[duplicate_queries["count"] > 1].copy()

print(f"📋 Duplicate Queries Analysis:")
print(f"=============================")
print(f"Queries with duplicates: {len(duplicates_only)}")
print(
    f"Total duplicate records: {duplicates_only['count'].sum() - len(duplicates_only)}"
)

# Show first few duplicates
print(f"\n🔍 First 5 Duplicate Queries:")
print(f"==============================")
for idx, row in duplicates_only.head().iterrows():
    print(
        f"\nQuery: {row['query_content'][:80]}{'...' if len(row['query_content']) > 80 else ''}"
    )
    print(
        f"Count: {row['count']} | Keep ID: {row['first_id']} | All IDs: {row['all_ids']}"
    )

duplicates_only

📋 Duplicate Queries Analysis:
Queries with duplicates: 25
Total duplicate records: 25

🔍 First 5 Duplicate Queries:

Query: beef soup
Count: 2 | Keep ID: 425 | All IDs: [2050, 425]

Query: breakfast platter
Count: 2 | Keep ID: 462 | All IDs: [2131, 462]

Query: corn soup
Count: 2 | Keep ID: 642 | All IDs: [2499, 642]

Query: cream filled cookies
Count: 2 | Keep ID: 653 | All IDs: [2520, 653]

Query: egg rice bowl
Count: 2 | Keep ID: 719 | All IDs: [2642, 719]


,query_content,count,first_id,all_ids
227,beef soup,2,425,"[2050, 425]"
360,breakfast platter,2,462,"[2131, 462]"
978,corn soup,2,642,"[2499, 642]"
1013,cream filled cookies,2,653,"[2520, 653]"
1242,egg rice bowl,2,719,"[2642, 719]"
1378,fish with vegetables,2,753,"[2705, 753]"
1597,fruit platter,2,821,"[2842, 821]"
1600,fruit salad,2,822,"[2845, 822]"
1721,grilled chicken salad,2,858,"[2914, 858]"
1771,grilled salmon with vegetables,2,877,"[2946, 877]"


In [17]:
# Update example table to reassign duplicate query_ids to first instance
print("🔄 Updating example table to reassign duplicate query_ids...")

# Create mapping of duplicate IDs to first ID
id_mapping = {}
for _, row in duplicates_only.iterrows():
    first_id = row["first_id"]
    all_ids = row["all_ids"]
    # Map all duplicate IDs to the first ID
    for query_id in all_ids:
        if query_id != first_id:  # Don't map the first ID to itself
            id_mapping[query_id] = first_id

print(f"Created mapping for {len(id_mapping)} duplicate query IDs")

🔄 Updating example table to reassign duplicate query_ids...
Created mapping for 25 duplicate query IDs


In [18]:
id_mapping

{2050: 425,
 2131: 462,
 2499: 642,
 2520: 653,
 2642: 719,
 2705: 753,
 2842: 821,
 2845: 822,
 2914: 858,
 2946: 877,
 2969: 891,
 4976: 3112,
 3198: 1036,
 3327: 1116,
 3554: 1264,
 3651: 1320,
 3774: 1378,
 4024: 1555,
 4025: 1557,
 4048: 1569,
 4062: 1576,
 4132: 1613,
 4355: 1748,
 4483: 1821,
 4604: 1883}

In [19]:
# Update example table using the mapping
updates_made = 0
with get_db_connection() as conn:
    with conn.cursor() as cursor:
        for old_id, new_id in id_mapping.items():
            # Update examples that reference the duplicate query_id
            cursor.execute(
                """
                UPDATE example 
                SET query_id = %s 
                WHERE query_id = %s
            """,
                (new_id, old_id),
            )

            updated_rows = cursor.rowcount
            updates_made += updated_rows

            if updated_rows > 0:
                print(f"Updated {updated_rows} examples: query_id {old_id} → {new_id}")

        # Commit all updates
        conn.commit()

print(f"\n✅ Updated {updates_made} example records")
print(f"📋 Ready to delete {len(id_mapping)} duplicate queries")

Updated 1 examples: query_id 2050 → 425
Updated 1 examples: query_id 2131 → 462
Updated 1 examples: query_id 2499 → 642
Updated 1 examples: query_id 2520 → 653
Updated 1 examples: query_id 2642 → 719
Updated 1 examples: query_id 2705 → 753
Updated 1 examples: query_id 2842 → 821
Updated 1 examples: query_id 2845 → 822
Updated 1 examples: query_id 2914 → 858
Updated 1 examples: query_id 2946 → 877
Updated 1 examples: query_id 2969 → 891
Updated 2 examples: query_id 4976 → 3112
Updated 1 examples: query_id 3198 → 1036
Updated 1 examples: query_id 3327 → 1116
Updated 1 examples: query_id 3554 → 1264
Updated 1 examples: query_id 3651 → 1320
Updated 1 examples: query_id 3774 → 1378
Updated 1 examples: query_id 4024 → 1555
Updated 1 examples: query_id 4025 → 1557
Updated 1 examples: query_id 4048 → 1569
Updated 1 examples: query_id 4062 → 1576
Updated 1 examples: query_id 4132 → 1613
Updated 1 examples: query_id 4355 → 1748
Updated 2 examples: query_id 4483 → 1821
Updated 2 examples: query_i

In [21]:
# Delete duplicate queries from query table (keeping only first instance)
print("🗑️  Deleting duplicate queries from query table...")

duplicate_ids_to_delete = list(id_mapping.keys())
print(f"Deleting {len(duplicate_ids_to_delete)} duplicate query records")
print(f"Sample IDs to delete: {duplicate_ids_to_delete}")

🗑️  Deleting duplicate queries from query table...
Deleting 25 duplicate query records
Sample IDs to delete: [2050, 2131, 2499, 2520, 2642, 2705, 2842, 2845, 2914, 2946, 2969, 4976, 3198, 3327, 3554, 3651, 3774, 4024, 4025, 4048, 4062, 4132, 4355, 4483, 4604]


In [22]:
deleted_count = 0
with get_db_connection() as conn:
    with conn.cursor() as cursor:
        # Delete duplicate queries in batches
        batch_size = 100
        for i in range(0, len(duplicate_ids_to_delete), batch_size):
            batch = duplicate_ids_to_delete[i : i + batch_size]

            # Create placeholders for the batch
            placeholders = ",".join(["%s"] * len(batch))

            cursor.execute(
                f"""
                DELETE FROM query 
                WHERE id IN ({placeholders})
            """,
                batch,
            )

            batch_deleted = cursor.rowcount
            deleted_count += batch_deleted
            print(f"Deleted batch {i // batch_size + 1}: {batch_deleted} queries")

        # Commit all deletions
        conn.commit()

print(f"\n✅ Successfully deleted {deleted_count} duplicate queries")
print(f"🎯 Kept {len(duplicates_only)} unique queries (first instances)")

Deleted batch 1: 25 queries

✅ Successfully deleted 25 duplicate queries
🎯 Kept 25 unique queries (first instances)


In [23]:
# Verify deduplication completed successfully
print("🔍 Verifying deduplication results...")

# Re-read query data to verify
with get_db_connection() as conn:
    # Get updated query counts
    updated_query_df = pd.read_sql(
        """
        SELECT 
            id,
            query_content,
            created_at
        FROM query
        ORDER BY created_at DESC
    """,
        conn,
    )

    # Check for any remaining duplicates
    remaining_duplicates = updated_query_df["query_content"].value_counts()
    remaining_duplicates = remaining_duplicates[remaining_duplicates > 1]

    # Verify example table integrity
    orphaned_examples = pd.read_sql(
        """
        SELECT COUNT(*) as orphaned_count
        FROM example e
        LEFT JOIN query q ON e.query_id = q.id
        WHERE q.id IS NULL
    """,
        conn,
    )

print(f"\n📊 Final Verification Results:")
print(f"=============================")
print(f"Total queries after deduplication: {len(updated_query_df):,}")
print(
    f"Unique queries after deduplication: {updated_query_df['query_content'].nunique():,}"
)
print(f"Remaining duplicates: {len(remaining_duplicates)}")
print(f"Orphaned examples: {orphaned_examples.iloc[0]['orphaned_count']}")

if len(remaining_duplicates) == 0 and orphaned_examples.iloc[0]["orphaned_count"] == 0:
    print(f"\n✅ SUCCESS: Deduplication completed successfully!")
    print(f"   - No duplicate queries remain")
    print(f"   - No orphaned examples found")
    print(f"   - Removed {total_records - len(updated_query_df)} duplicate records")
else:
    print(f"\n⚠️  WARNING: Issues detected:")
    if len(remaining_duplicates) > 0:
        print(f"   - {len(remaining_duplicates)} queries still have duplicates")
    if orphaned_examples.iloc[0]["orphaned_count"] > 0:
        print(
            f"   - {orphaned_examples.iloc[0]['orphaned_count']} orphaned examples found"
        )

🔍 Verifying deduplication results...


/var/folders/jk/dyw0vdnx2jg9lyq8m01n8nfm0000gn/T/ipykernel_80367/4240732941.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  updated_query_df = pd.read_sql("""



📊 Final Verification Results:
Total queries after deduplication: 4,932
Unique queries after deduplication: 4,932
Remaining duplicates: 0
Orphaned examples: 0

✅ SUCCESS: Deduplication completed successfully!
   - No duplicate queries remain
   - No orphaned examples found
   - Removed 25 duplicate records


/var/folders/jk/dyw0vdnx2jg9lyq8m01n8nfm0000gn/T/ipykernel_80367/4240732941.py:21: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  orphaned_examples = pd.read_sql("""
